## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [15:31,  8.05it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7500/7500 [00:39<00:00, 189.51it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Insurance Fraud Enforcement Department,City of London,We’re the @CityPoliceFraud unit that tackles U...,2012-02-03 13:57:48,3707,547,291,True,2020-08-12 09:49:46,Criminals will use whatever means they can to ...,None,Twitter Web App,False
0,AmigaRules 🇦🇺🇬🇧 #DefundBBC #BREXIT2021,United Kingdom,"""The internet is good at bringing people toget...",2013-05-12 08:43:32,233,365,1290,False,2020-08-12 09:49:46,"@piersmorgan Ah you might say that Piers,\n\nB...",None,Twitter Web App,False
0,Novel Coronavirus - Covid19,,"Global Coronavirus news from journals, media, ...",2020-01-06 17:09:38,44435,36,6,False,2020-08-12 09:49:45,Association Between Statewide School Closure a...,[COVID19],Twitter Web App,False
0,John Cornelius,,"Languages, the arts, politics & current affair...",2015-06-13 19:51:12,174,269,8592,False,2020-08-12 09:49:45,I suspect we’re going to be seeing a lot of th...,[Covid19],Twitter for iPad,False
0,MochaParties,Universal,#Lifestyle events with a NO PRESSURE ENVIRONME...,2010-07-16 05:10:54,2396,1836,4,False,2020-08-12 09:49:44,The latest The Interracial Swingers Daily! htt...,"[technology, covid19]",Paper.li,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (110101, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 110101 all tweets: 117601


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (117601, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)